In [1]:
# Import libraries

import smartsheet
import pandas as pd
import re
import sqlite3

In [18]:
#Access to smartsheet API

mytoken = ''
sheet_id= ''

ss = smartsheet.Smartsheet(mytoken)
ss.errors_as_exceptions(True)
mysheet = ss.Sheets.get_sheet(sheet_id)

i = 0
for column in mysheet.columns:
    
    print(i, column.title)
    i+=1

0 Customer
1 Project Name
2 Create Date
3 Reference Image
4 Sales Rep
5 US PM
6 China PM
7 Customer PO No.
8 Impact SO No.
9 Impact PO No.
10 Supplier
11 Date Samples Will Be Ready In China
12 Art File Received
13 Signed File / CAD Approved
14 Approved Signed File Received
15 PPS Approved
16 MP Completion Date
17 Due Date To Customer
18 Shipped Via
19 FREIGHT BOOKING INFO
20 FOB POINT
21 Structural Sample
22 Signed File or CAD (Clamshell) Approved
23 Quantity
24 Status
25 Customer PO Date
26 IN HOUSE DUE DATE
27 Photo of project
28 Request Date
29 Project No.
30 Priority
31 Suppliers
32 Health
33 Last Modified Date
34 Column47
35 Latest Comment
36 Column49
37 Column31
38 Column51
39 Signed File Approved
40 Column16
41 Column36
42 Column37
43 Column39
44 Column40
45 Column41
46 Column42
47 Column43
48 Column44
49 Column45
50 No of SKUs
51 Column46
52 Column48
53 Pkg Category
54 Quoted Qty
55 Overage
56 Shipping Info (Incoterm/City/State/ZIP)
57 Stage
58 Milestone
59 Reason Code
60 1st R

In [19]:
#Get the list of columns that need to be imported

import_list=['Customer', 'Project Name','China PM','Customer PO No.','FREIGHT BOOKING INFO','Status', 'Last Modified Date']
col_d= {}

for item in import_list:
    col_d[item] = []

pos_list =[]

for i in range(len(mysheet.columns)):
    if mysheet.columns[i].title in col_d:
        pos_list.append(i)

print(pos_list)


[0, 1, 6, 7, 19, 24, 33]


In [20]:
#Import smartsheet info to Dataframe 

for i in range(len(mysheet.rows)):
    if mysheet.rows[i].cells[5].value == "":
        for p in pos_list:
            col_d[mysheet.columns[p].title].append(mysheet.rows[i].cells[p].value)
            
df = pd.DataFrame(data=col_d)

df

,Customer,Project Name,China PM,Customer PO No.,FREIGHT BOOKING INFO,Status,Last Modified Date
0,Company A,Tray Tooling,Cyan,HP343432,None,"MZ 2/11/25: PO sent to ABC, customer FedEX# pr...",2025-04-28T01:08:46Z
1,Company B,paper bag,Queenie,BAG46565,Loaded on 2/21\nContainer No.: OOLU1877706\nVE...,MZ1/8/25 PO sent to BCD\nJack(3/3):Products sh...,2025-05-04T17:49:03Z
2,Company C,Kraft paper bags,Shine,PO8347364,None,MZ 2/28/25: Customer approved sample\nMZ 3/3/2...,2025-04-28T01:08:46Z
3,Company D,Paper bag + Sample,Queenie,SIGNED434343,None,MZ 2/28/25: Sample approved\nJack(3/3):waiting...,2025-05-04T17:49:03Z
4,Company E,Shopping paper bag,Queenie,PO3436,None,MZ 2/2725: PO sent to ABC\nJack(3/3):waiting f...,2025-04-28T01:08:46Z
5,Company F,Plastic Tray A & Tray B,Queenie,F0011600,None,MZ 1/14/25 :PO sent to BCD\n2/21/25 MZ: Shipme...,2025-04-28T01:08:46Z
6,Company G,NYT95007 production Tooling,June,T192732,None,MZ 1/8/25: PO sent to CDE\nLinda2/24: Products...,2025-04-28T01:08:46Z
7,Company G,NYT95132-001 Tooling,June,PO-1939264,None,MZ 2/7/25: PO sent to ABC\nLinda2/24: The draw...,2025-05-04T17:49:03Z
8,Company G,"Sample, Shipper for NYT95112",June,None,None,Linda3/17: Waiting for new PO.,2025-04-28T00:09:47Z
9,Company K,2025 Series,June,None,None,None,2025-04-28T00:09:47Z


In [21]:
#Prepare for the project status dataframe 

df2 = df.drop(columns=['Customer', 'Project Name', 'China PM', 'Customer PO No.', 'FREIGHT BOOKING INFO', 'Last Modified Date'])

df2["Status"] = df2["Status"].str.split("\n")
df2 = df2.explode("Status")

pattern = r'(\d{1,2}\/\d{1,2}\/\d{0,2}|\d{1,2}\/\d{1,2})'

df2['Date'] = df2['Status'].str.extract(pattern, expand = False)

df2

,Status,Date
0,"MZ 2/11/25: PO sent to ABC, customer FedEX# pr...",2/11/25
0,MZ 2/19: Samples will be ready on 3/1/25,2/19
0,Cheryl 3/1: prototype will be delay to 3/4,3/1
0,Cheryl 3/6: Tracking number is 7777777779(FedEx),3/6
1,MZ1/8/25 PO sent to BCD,1/8/25
...,...,...
18,800-0000 - Air shipment via Fedex#777777 on 3/3.,3/3
18,"400-1111 - Ocean shipment, plan to load this w...",NaN
18,Loaded on 3/8,3/8
18,Container#OOCU88888888,NaN


In [22]:
#Split each status message from the status column

df2['msg'] = df2['Status'].str.replace(pattern, '', regex=True, n=1)

df2 = df2.dropna()

df2 = df2.reset_index()

#Add id as index in Django
df2['id'] = df2.index
df2

,index,Status,Date,msg,id
0,0,"MZ 2/11/25: PO sent to ABC, customer FedEX# pr...",2/11/25,"MZ : PO sent to ABC, customer FedEX# provided.",0
1,0,MZ 2/19: Samples will be ready on 3/1/25,2/19,MZ : Samples will be ready on 3/1/25,1
2,0,Cheryl 3/1: prototype will be delay to 3/4,3/1,Cheryl : prototype will be delay to 3/4,2
3,0,Cheryl 3/6: Tracking number is 7777777779(FedEx),3/6,Cheryl : Tracking number is 7777777779(FedEx),3
4,1,MZ1/8/25 PO sent to BCD,1/8/25,MZ PO sent to BCD,4
5,1,Jack(3/3):Products shipped. Thanks.,3/3,Jack():Products shipped. Thanks.,5
6,2,MZ 2/28/25: Customer approved sample,2/28/25,MZ : Customer approved sample,6
7,2,MZ 3/3/25: MP ready 4/10,3/3/25,MZ : MP ready 4/10,7
8,3,MZ 2/28/25: Sample approved,2/28/25,MZ : Sample approved,8
9,3,Jack(3/3):waiting for approval on Sample.Thanks.,3/3,Jack():waiting for approval on Sample.Thanks.,9


In [23]:
#Extract Date from each message

pattern2 = r'(^\d{1,2}\/\d{1,2}$)'

df2['Date2'] = df2['Date'].str.extract(pattern2, expand=False)

df2['Date3'] = pd.to_datetime(df2['Date2'], format='%m/%d')

df2['Date4'] = df2['Date'].str.replace(pattern2, '', regex=True)

df2['Date4'] = pd.to_datetime(df2['Date4'], format='%m/%d/%y')

df2

,index,Status,Date,msg,id,Date2,Date3,Date4
0,0,"MZ 2/11/25: PO sent to ABC, customer FedEX# pr...",2/11/25,"MZ : PO sent to ABC, customer FedEX# provided.",0,NaN,NaT,2025-02-11
1,0,MZ 2/19: Samples will be ready on 3/1/25,2/19,MZ : Samples will be ready on 3/1/25,1,2/19,1900-02-19,NaT
2,0,Cheryl 3/1: prototype will be delay to 3/4,3/1,Cheryl : prototype will be delay to 3/4,2,3/1,1900-03-01,NaT
3,0,Cheryl 3/6: Tracking number is 7777777779(FedEx),3/6,Cheryl : Tracking number is 7777777779(FedEx),3,3/6,1900-03-06,NaT
4,1,MZ1/8/25 PO sent to BCD,1/8/25,MZ PO sent to BCD,4,NaN,NaT,2025-01-08
5,1,Jack(3/3):Products shipped. Thanks.,3/3,Jack():Products shipped. Thanks.,5,3/3,1900-03-03,NaT
6,2,MZ 2/28/25: Customer approved sample,2/28/25,MZ : Customer approved sample,6,NaN,NaT,2025-02-28
7,2,MZ 3/3/25: MP ready 4/10,3/3/25,MZ : MP ready 4/10,7,NaN,NaT,2025-03-03
8,3,MZ 2/28/25: Sample approved,2/28/25,MZ : Sample approved,8,NaN,NaT,2025-02-28
9,3,Jack(3/3):waiting for approval on Sample.Thanks.,3/3,Jack():waiting for approval on Sample.Thanks.,9,3/3,1900-03-03,NaT


In [24]:
#Fix date format

df2['Date3'] = df2['Date3'] + pd.DateOffset(years=125)
df2

,index,Status,Date,msg,id,Date2,Date3,Date4
0,0,"MZ 2/11/25: PO sent to ABC, customer FedEX# pr...",2/11/25,"MZ : PO sent to ABC, customer FedEX# provided.",0,NaN,NaT,2025-02-11
1,0,MZ 2/19: Samples will be ready on 3/1/25,2/19,MZ : Samples will be ready on 3/1/25,1,2/19,2025-02-19,NaT
2,0,Cheryl 3/1: prototype will be delay to 3/4,3/1,Cheryl : prototype will be delay to 3/4,2,3/1,2025-03-01,NaT
3,0,Cheryl 3/6: Tracking number is 7777777779(FedEx),3/6,Cheryl : Tracking number is 7777777779(FedEx),3,3/6,2025-03-06,NaT
4,1,MZ1/8/25 PO sent to BCD,1/8/25,MZ PO sent to BCD,4,NaN,NaT,2025-01-08
5,1,Jack(3/3):Products shipped. Thanks.,3/3,Jack():Products shipped. Thanks.,5,3/3,2025-03-03,NaT
6,2,MZ 2/28/25: Customer approved sample,2/28/25,MZ : Customer approved sample,6,NaN,NaT,2025-02-28
7,2,MZ 3/3/25: MP ready 4/10,3/3/25,MZ : MP ready 4/10,7,NaN,NaT,2025-03-03
8,3,MZ 2/28/25: Sample approved,2/28/25,MZ : Sample approved,8,NaN,NaT,2025-02-28
9,3,Jack(3/3):waiting for approval on Sample.Thanks.,3/3,Jack():waiting for approval on Sample.Thanks.,9,3/3,2025-03-03,NaT


In [25]:
#Trim dataframe

df2['Date4'] = df2['Date4'].fillna(df2['Date3'])

df4 = df2.drop(columns=['Status', 'Date', 'Date2', 'Date3'])
df4

,index,msg,id,Date4
0,0,"MZ : PO sent to ABC, customer FedEX# provided.",0,2025-02-11
1,0,MZ : Samples will be ready on 3/1/25,1,2025-02-19
2,0,Cheryl : prototype will be delay to 3/4,2,2025-03-01
3,0,Cheryl : Tracking number is 7777777779(FedEx),3,2025-03-06
4,1,MZ PO sent to BCD,4,2025-01-08
5,1,Jack():Products shipped. Thanks.,5,2025-03-03
6,2,MZ : Customer approved sample,6,2025-02-28
7,2,MZ : MP ready 4/10,7,2025-03-03
8,3,MZ : Sample approved,8,2025-02-28
9,3,Jack():waiting for approval on Sample.Thanks.,9,2025-03-03


In [26]:
#Clean column name for SQL Database

df3 = df.drop(columns=['Status'])
df3 = df3.rename(columns={'Project Name':'Project', 'China PM':'PMCN', 'Customer PO No.':'PO', 'FREIGHT BOOKING INFO':'Booking', 'Last Modified Date':'Last_update'})
df3

,Customer,Project,PMCN,PO,Booking,Last_update
0,Company A,Tray Tooling,Cyan,HP343432,None,2025-04-28T01:08:46Z
1,Company B,paper bag,Queenie,BAG46565,Loaded on 2/21\nContainer No.: OOLU1877706\nVE...,2025-05-04T17:49:03Z
2,Company C,Kraft paper bags,Shine,PO8347364,None,2025-04-28T01:08:46Z
3,Company D,Paper bag + Sample,Queenie,SIGNED434343,None,2025-05-04T17:49:03Z
4,Company E,Shopping paper bag,Queenie,PO3436,None,2025-04-28T01:08:46Z
5,Company F,Plastic Tray A & Tray B,Queenie,F0011600,None,2025-04-28T01:08:46Z
6,Company G,NYT95007 production Tooling,June,T192732,None,2025-04-28T01:08:46Z
7,Company G,NYT95132-001 Tooling,June,PO-1939264,None,2025-05-04T17:49:03Z
8,Company G,"Sample, Shipper for NYT95112",June,None,None,2025-04-28T00:09:47Z
9,Company K,2025 Series,June,None,None,2025-04-28T00:09:47Z


In [27]:
#Convert datetime format and add id as index

df3['Last_update'] = pd.to_datetime(df3['Last_update'])

#important, need to add id as index as Django use this format
df3['id'] = df.index

df3

,Customer,Project,PMCN,PO,Booking,Last_update,id
0,Company A,Tray Tooling,Cyan,HP343432,None,2025-04-28 01:08:46+00:00,0
1,Company B,paper bag,Queenie,BAG46565,Loaded on 2/21\nContainer No.: OOLU1877706\nVE...,2025-05-04 17:49:03+00:00,1
2,Company C,Kraft paper bags,Shine,PO8347364,None,2025-04-28 01:08:46+00:00,2
3,Company D,Paper bag + Sample,Queenie,SIGNED434343,None,2025-05-04 17:49:03+00:00,3
4,Company E,Shopping paper bag,Queenie,PO3436,None,2025-04-28 01:08:46+00:00,4
5,Company F,Plastic Tray A & Tray B,Queenie,F0011600,None,2025-04-28 01:08:46+00:00,5
6,Company G,NYT95007 production Tooling,June,T192732,None,2025-04-28 01:08:46+00:00,6
7,Company G,NYT95132-001 Tooling,June,PO-1939264,None,2025-05-04 17:49:03+00:00,7
8,Company G,"Sample, Shipper for NYT95112",June,None,None,2025-04-28 00:09:47+00:00,8
9,Company K,2025 Series,June,None,None,2025-04-28 00:09:47+00:00,9


In [28]:
#Connect to database

filepath = r"FILEPATH"
con = sqlite3.connect(filepath)


In [29]:
#Test database connection

cur = con.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('django_migrations',), ('sqlite_sequence',), ('auth_group_permissions',), ('auth_user_groups',), ('auth_user_user_permissions',), ('django_admin_log',), ('django_content_type',), ('auth_permission',), ('auth_group',), ('auth_user',), ('django_session',), ('ss_testdj',), ('ss_status',)]


In [30]:
#export dataframe to database

df3.to_sql(name ='ss_testdj', con = con, if_exists='replace', index=True)


19

In [31]:
#verify databse

cur.execute(f"SELECT * FROM ss_testdj")
rows = cur.fetchall()

if not rows:
    print(f"Table '{table_name}' is empty or does not exist.")

# Print header
column_names = [description[0] for description in cur.description]
print("|".join(f" {name} " for name in column_names))
print("-" * (sum(len(name) + 2 for name in column_names) + len(column_names) - 1))

# Print rows
for row in rows:
    print("|".join(f" {str(value)} " for value in row))

 index | Customer | Project | PMCN | PO | Booking | Last_update | id 
---------------------------------------------------------------------
 0 | Company A | Tray Tooling | Cyan | HP343432 | None | 2025-04-28 01:08:46+00:00 | 0 
 1 | Company B | paper bag | Queenie | BAG46565 | Loaded on 2/21
Container No.: OOLU1877706
VESSEL:   COSCO SPAIN 070E
ETD: 27 Feb 202
ETA to Harbor:  18 Mar 2025 | 2025-05-04 17:49:03+00:00 | 1 
 2 | Company C | Kraft paper bags | Shine | PO8347364 | None | 2025-04-28 01:08:46+00:00 | 2 
 3 | Company D | Paper bag + Sample | Queenie | SIGNED434343 | None | 2025-05-04 17:49:03+00:00 | 3 
 4 | Company E | Shopping paper bag | Queenie | PO3436 | None | 2025-04-28 01:08:46+00:00 | 4 
 5 | Company F | Plastic Tray A & Tray B | Queenie | F0011600 | None | 2025-04-28 01:08:46+00:00 | 5 
 6 | Company G | NYT95007 production Tooling | June | T192732 | None | 2025-04-28 01:08:46+00:00 | 6 
 7 | Company G | NYT95132-001 Tooling | June | PO-1939264 | None | 2025-05-04 17:4

In [32]:
#Export status message dataframe to database

df4.to_sql(name ='ss_status', con = con, if_exists='replace', index=True)

cur.execute(f"SELECT * FROM ss_status")
rows = cur.fetchall()

if not rows:
    print(f"Table '{table_name}' is empty or does not exist.")

# Print header
column_names = [description[0] for description in cur.description]
print("|".join(f" {name} " for name in column_names))
print("-" * (sum(len(name) + 2 for name in column_names) + len(column_names) - 1))

# Print rows
for row in rows:
    print("|".join(f" {str(value)} " for value in row))

 level_0 | index | msg | id | Date4 
------------------------------------
 0 | 0 | MZ : PO sent to ABC, customer FedEX# provided. | 0 | 2025-02-11 00:00:00 
 1 | 0 | MZ : Samples will be ready on 3/1/25 | 1 | 2025-02-19 00:00:00 
 2 | 0 | Cheryl : prototype will be delay to 3/4 | 2 | 2025-03-01 00:00:00 
 3 | 0 | Cheryl : Tracking number is 7777777779(FedEx) | 3 | 2025-03-06 00:00:00 
 4 | 1 | MZ PO sent to BCD | 4 | 2025-01-08 00:00:00 
 5 | 1 | Jack():Products shipped. Thanks. | 5 | 2025-03-03 00:00:00 
 6 | 2 | MZ : Customer approved sample | 6 | 2025-02-28 00:00:00 
 7 | 2 | MZ : MP ready 4/10 | 7 | 2025-03-03 00:00:00 
 8 | 3 | MZ : Sample approved | 8 | 2025-02-28 00:00:00 
 9 | 3 | Jack():waiting for approval on Sample.Thanks. | 9 | 2025-03-03 00:00:00 
 10 | 3 | MZ : ABC will have MP done 3/25 | 10 | 2025-03-04 00:00:00 
 11 | 4 | MZ 25: PO sent to ABC | 11 | 2025-02-27 00:00:00 
 12 | 4 | Jack():waiting for product. | 12 | 2025-03-03 00:00:00 
 13 | 4 | MZ : Products ready 3/1

In [17]:
#Option delete table

cur.execute(f'DROP TABLE IF EXISTS ss_stauts')

In [ ]:
#Option delete table
cur.execute(f'DROP TABLE IF EXISTS ss_status')

In [33]:
#Close database connection

con.close() 